In [159]:
import numpy as np
from numpy import linalg as LA
import pandas as pd

from string import punctuation
import re
import nltk
from nltk.stem.porter import PorterStemmer
p_stemmer = PorterStemmer()
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
import itertools
from collections import defaultdict
import streamlit as st

In [160]:
# reads the file
df = pd.read_csv("walmart-products-modified.csv", encoding='latin1')

In [161]:
df.head() 

,name,rating,type_of,price,comments
0,"Eliminator Ant, Roach & Spider Killer4, 20 oz,...",4.0840,"Pest Bait, Traps & Killers",3.48,Covered one up in phone. The roach crawled out...
1,"Spiderman Party Blowers, 8ct",NaN,Novelty & Gag Toys,3.74,NaN
2,"TERRO Spider and Ant Killer Aerosol Spray, 16 ...",4.5333,"Pest Bait, Traps & Killers",4.97,EXTREMELY disappointed. Raid worked way faster...
3,"Hot Shot Fresh Floral Scent Ant, Roach & Spide...",4.1032,"Pest Bait, Traps & Killers",6.36,Sprayed them directly 10 times. did nothing!
4,"Eliminator Ant, Roach & Spider Killer4, Aeroso...",4.3548,"Pest Bait, Traps & Killers",5.98,It does somewhat kill on contact but it's real...


In [162]:
df.shape[0]

862

In [163]:
# the different zones (tables) to create the inverted-indexes
terms_dictionary_for_zone = {
        'name': { },
         'type_of': { },
         'comments': { }
         }

In [164]:
def stemmed_inverted_index_from_data(
    
    
    # the dataframe
    doc_database # list[str,]
):
    terms_dictionary_for_zone = {
            'name': { },
            'type_of': { },
            'comments': { }
            }
    for zone in doc_database.columns:
        # only doing indexing for the chosen zones above
        if zone in terms_dictionary_for_zone.keys():
            # saving docID and the row content for the chosen zones
            for docID, doc in enumerate(df[zone]): # int, str; list[(int, str), ]
                
                # tokenises the words in the row of the selected zone based on NLTK vocab
                sentence_doc = word_tokenize(str(doc)) # list[str,];
                # stems all words in doc
                sentence_str_1 = [p_stemmer.stem(word_tk) for word_tk in sentence_doc]  # list[str,]
                # joins the stemmed tokens back into a sentence for further procesing
                sentence_str_1 = " ".join(sentence_str_1) # str

                # subs all symbols/punctuations (including space) with a space (' '); excludes digits
                sentence_str_1 = re.sub(r'\W+', ' ',sentence_str_1)
                # subs all digits with a space (' '); a sequence of digits (without space) is given one (' ')
                sentence_str_1 = re.sub(r'\d+', ' ',sentence_str_1)
                # subs all symbols AND digits (including spaces) with a space (' '); includes digits
                sentence_str_1 = re.sub(r'[^a-zA-Z]', ' ',sentence_str_1)
                # compiles a pattern and subs out the pattern in sentence_str_1 with ' '
                sentence_str_1 = re.compile('(\s*)pron(\s*)').sub(' ', sentence_str_1)

                # tokenize to clean up one more time if 're' has introduced and garbage
                sentence_doc_1 = word_tokenize(sentence_str_1) # list[str,]
                # strip removes whitespace at beginning and end of string (not in between)
                sentence_str_2 = [str(word_tk_1).strip() for word_tk_1 in sentence_doc_1] # list[str,]
                # strings with >=2 chars only
                sentence_str_2  = [word_str for word_str in sentence_str_2 if len(word_str) >= 2] # list[str,]
                # strings which are not symbols/punctuations
                sentence_str_2 = [word_str for word_str in sentence_str_2  if word_str not in punctuation]
                # strings which are not stop-words; check with: Spacy_Eng.vocab['word'].is_stop
                sentence_str_2 = [word_str for word_str in sentence_str_2 if word_str not in stop_words]

                for term in sentence_str_2: # 'str'
                    # for each zone the term increased in doc freq and appends the posting list if term exists
                        if term in terms_dictionary_for_zone[zone]: # previously from this doc or other docs
                            # print(term, terms_dictionary_for_zone[zone], type(terms_dictionary_for_zone[zone][term][1]))
                            if docID in terms_dictionary_for_zone[zone][term][1].keys(): # previously from this doc
                                org_val = terms_dictionary_for_zone[zone][term][1][docID]
                                terms_dictionary_for_zone[zone][term][1].update({docID : org_val+1})
                            else: # from other docs
                                terms_dictionary_for_zone[zone][term][1].update({docID : 1})
                                terms_dictionary_for_zone[zone][term][0] += 1
                                

                                
                            # terms_dictionary_for_zone[zone][term][1].append([docID, ]) # list[int,]
                            # if docID not in terms_dictionary_for_zone[zone][term][1].keys():
                            #     terms_dictionary_for_zone[zone][term][0] += 1 # int; doc freq
                        # if term doesnt exist, put doc freq as 1 and append the first posting list
                        else: # previously non-existant
                            terms_dictionary_for_zone[zone][term] =[[],{}] # list[[int], [int, ]]; list[[doc freq], [docIds,]]
                            terms_dictionary_for_zone[zone][term][0] = 1 # int; doc freq
                            terms_dictionary_for_zone[zone][term][1].update({docID : 1})
                            # print(type(terms_dictionary_for_zone[zone][term][1]))
    # return the inverted-index dictionary
    return terms_dictionary_for_zone

In [165]:
# # Test case
# doc1 = {
#     'name': 'Fuji',
#     'rating': '4.5',
#     'type_of': 'apple',
#     'category': 'fruit',
#     'price': '2.50',
#     'comments': 'this is an apple, it is kind of fresh but also small, smaller than a grape'
# }
# doc2 = {
#     'name': 'mcIntosh',
#     'rating': '3.6',
#     'type_of': 'apple',
#     'category': 'fruit',
#     'price': '3.50',
#     'comments': 'it was not as sweet as i hoped'
# }
# doc3 = {
#     'name': 'concord',
#     'rating': '4.8',
#     'type_of': 'grape',
#     'category': 'fruit',
#     'price': '4.50',
#     'comments': 'trying grapes for the first time, not too bad'
# }
# doc4 = {
#     'name': 'Red anjou',
#     'rating': '4.5',
#     'type_of': 'pear',
#     'category': 'fruit',
#     'price': '1.50',
#     'comments': 'the shape of the peach is appealing, haha'
# }


# doc_database_list = [doc1, doc2, doc3, doc4] # list[dict, ]

# doc_database = defaultdict(list) # defaultdict({key: list[str, ]})
# for doc in doc_database_list:
#     for key, val in doc.items():
#         doc_database[key].append(val)
# doc_database

# my_dict = {}
# for doc in doc_database_list:
#     for key, val in doc.items():
#         if key not in my_dict.keys():
#             my_dict.update({key: [val]})
#         else:
#             my_dict[key].append(val)
# my_dict





In [166]:
inverted_index_zonal_dictionary = stemmed_inverted_index_from_data(df)


In [167]:
len(inverted_index_zonal_dictionary['name'].keys())

1275

In [168]:
def query_processing(
    query # str
):
    posting_list = [] # list[]
    # split long string of query terms into into different individual terms based on whitespace
    # query_terms = query.split() #list[str, ]
    # each_term = 0 # int
                
    # tokenises the words in the row of the selected zone based on NLTK vocab
    sentence_doc = word_tokenize(str(query)) # list[str,];
    # stems all words in doc
    sentence_str_1 = [p_stemmer.stem(word_tk) for word_tk in sentence_doc]  # list[str,]
    # joins the stemmed tokens back into a sentence for further procesing
    sentence_str_1 = " ".join(sentence_str_1) # str

    # subs all symbols/punctuations (including space) with a space (' '); excludes digits
    sentence_str_1 = re.sub(r'\W+', ' ',sentence_str_1)
    # subs all digits with a space (' '); a sequence of digits (without space) is given one (' ')
    sentence_str_1 = re.sub(r'\d+', ' ',sentence_str_1)
    # subs all symbols AND digits (including spaces) with a space (' '); includes digits
    sentence_str_1 = re.sub(r'[^a-zA-Z]', ' ',sentence_str_1)
    # compiles a pattern and subs out the pattern in sentence_str_1 with ' '
    sentence_str_1 = re.compile('(\s*)pron(\s*)').sub(' ', sentence_str_1)

    # tokenize to clean up one more time if 're' has introduced and garbage
    sentence_doc_1 = word_tokenize(sentence_str_1) # list[str,]
    # strip removes whitespace at beginning and end of string (not in between)
    sentence_str_2 = [str(word_tk_1).strip() for word_tk_1 in sentence_doc_1] # list[str,]
    # strings with >=2 chars only
    sentence_str_2  = [word_str for word_str in sentence_str_2 if len(word_str) >= 2] # list[str,]
    # strings which are not symbols/punctuations
    sentence_str_2 = [word_str for word_str in sentence_str_2  if word_str not in punctuation]
    # strings which are not stop-words; check with: Spacy_Eng.vocab['word'].is_stop
    sentence_str_2 = [word_str for word_str in sentence_str_2 if word_str not in stop_words]

    
    for term in sentence_str_2[:]: # str       
        # for individual terms compute edit distance with each term in the 'name' inverted-index dictionary
        # this is useful if there is a typo in one or multiple of the query terms
        edit_dist_list = [[key, nltk.edit_distance(term, key)]  for key in inverted_index_zonal_dictionary['name']] # list[[str, double], ]
        edit_dist_list_1 = list(zip(*edit_dist_list)) # list[(str,)(double,)]
        # choose the min edit distance as the closest term match in the inverted-index dictionary
        if min(edit_dist_list_1[1]) >=2:
            print('No matching results')
            return 0


        internal_query = edit_dist_list_1[0][edit_dist_list_1[1].index(min(edit_dist_list_1[1]))] # str


        # from the term matched in the inverted-index, append, its posting list to 'posting-list'
        posting_list.append(inverted_index_zonal_dictionary['name'][internal_query][1]) # list[[int, ], ]
        # each_term +=1

    return posting_list

In [169]:
def tf_df(posting_list):
    
    # for multiple query terms, we also get (OR query)
    posting_list_combined = set(list(itertools.chain(*posting_list))) # list[int, ]
    # print(posting_list_combined)
    # posting_list_combined_for_or_low_priority = set(posting_list_combined) # set[int, ]


    ### tf|term,doc
    term_freq_given_doc_term = {}
    for docID in posting_list_combined:
        for i in range(0, len(posting_list), 1):
            if docID not in term_freq_given_doc_term.keys():
                if docID not in posting_list[i]:
                    term_freq_given_doc_term.update({docID : [0]})
                else:
                    term_freq_given_doc_term.update({docID : [posting_list[i][docID]]})
            else:
                if docID not in posting_list[i]:
                    term_freq_given_doc_term[docID].append(0)
                else:
                    term_freq_given_doc_term[docID].append(posting_list[i][docID])
    
    
    ### df|term
    doc_freq_given_terms = []
    for i in range(0, len(posting_list), 1):
        doc_freq_given_terms.append(len(posting_list[i]))


    return term_freq_given_doc_term, doc_freq_given_terms

In [170]:
def doc_scoring(term_f, doc_f):
    score = {}
    doc_score = {}
    for key, val in term_f.items():
        for i in range(0, len(val), 1):
            if key not in score.keys():
                tf_idf = (1 + np.log10(val[i])) * (np.log10(df.shape[0]/doc_f[i]))
                if tf_idf > -9999:
                    score.update({key : [tf_idf]})
                else:
                    score.update({key : [0]})
            else:
                tf_idf = (1 + np.log10(val[i])) * (np.log10(df.shape[0]/doc_f[i]))
                if tf_idf > -9999:
                    score[key].append(tf_idf)
                else:
                    score[key].append(0)
    
    for key, val in score.items():
        doc_score.update({key : sum(val)})


    return score, dict(sorted(doc_score.items(), key = lambda x : x[1], reverse= True))



In [171]:
query = 'eliminator ant' # 'str' typo; on both terms
pl= query_processing(query)
term_f, doc_f = tf_df(pl)
score, doc_score = doc_scoring(term_f,doc_f)
# print(term_f, doc_f)
print(doc_score)    


{0: 3.615742026546119, 4: 3.615742026546119, 82: 3.615742026546119, 797: 1.709555169685661, 129: 1.5057548917509769, 813: 1.5057548917509769, 307: 1.5057548917509769, 853: 1.5057548917509769, 2: 1.1573560154410691, 3: 1.1573560154410691, 5: 1.1573560154410691, 6: 1.1573560154410691, 7: 1.1573560154410691, 257: 1.1573560154410691, 265: 1.1573560154410691, 138: 1.1573560154410691, 139: 1.1573560154410691, 141: 1.1573560154410691, 14: 1.1573560154410691, 20: 1.1573560154410691, 151: 1.1573560154410691, 26: 1.1573560154410691, 283: 1.1573560154410691, 794: 1.1573560154410691, 30: 1.1573560154410691, 415: 1.1573560154410691, 674: 1.1573560154410691, 35: 1.1573560154410691, 163: 1.1573560154410691, 294: 1.1573560154410691, 557: 1.1573560154410691, 174: 1.1573560154410691, 430: 1.1573560154410691, 306: 1.1573560154410691, 563: 1.1573560154410691, 571: 1.1573560154410691, 449: 1.1573560154410691, 837: 1.1573560154410691, 332: 1.1573560154410691, 845: 1.1573560154410691, 336: 1.1573560154410691

C:\Users\Shiva\.conda\envs\nlp_env\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log10
  import sys


In [172]:
top_10_docID = list(doc_score.keys())[0:10]


In [173]:
df.loc[top_10_docID,:]

,name,rating,type_of,price,comments
0,"Eliminator Ant, Roach & Spider Killer4, 20 oz,...",4.0840,"Pest Bait, Traps & Killers",3.48,Covered one up in phone. The roach crawled out...
4,"Eliminator Ant, Roach & Spider Killer4, Aeroso...",4.3548,"Pest Bait, Traps & Killers",5.98,It does somewhat kill on contact but it's real...
82,"Eliminator Ant, Flea & Tick Killer Plus Granul...",4.0804,"Pest Bait, Traps & Killers",6.48,I bought it because I was expecting it to kill...
797,"TERRO T600 Ant Dust - Kills fire ants, carpent...",0.0000,"Pest Bait, Traps & Killers",18.49,NaN
129,"Ortho Home Defense Max Ant, Roach and Spider, ...",NaN,"Pest Bait, Traps & Killers",10.80,NaN
813,Disney Avengers Campus Spider-Bot Ant Man Wasp...,0.0000,Action Figures,44.99,NaN
307,Raid Ant & Roach Killer - Fragrance-Free - Spr...,0.0000,"Pest Bait, Traps & Killers",41.92,NaN
853,Disney Avengers Campus Spider-Bot Ant Man Tact...,0.0000,Action Figures,44.99,NaN
2,"TERRO Spider and Ant Killer Aerosol Spray, 16 ...",4.5333,"Pest Bait, Traps & Killers",4.97,EXTREMELY disappointed. Raid worked way faster...
3,"Hot Shot Fresh Floral Scent Ant, Roach & Spide...",4.1032,"Pest Bait, Traps & Killers",6.36,Sprayed them directly 10 times. did nothing!


PART 2 - Cosine similarity

In [174]:
df_2 = df

In [175]:
query_1 = df.loc[0, 'name']
query_1

'Eliminator Ant, Roach & Spider Killer4, 20 oz, Kills Insects & Spiders'

In [176]:
inverted_index_zonal_dictionary_second = stemmed_inverted_index_from_data(df_2)

In [177]:
len(inverted_index_zonal_dictionary_second['name'].keys())

1275

In [178]:
zero_data = np.zeros(shape=(1,len(inverted_index_zonal_dictionary_second['name'].keys())))
initiate_query_bow = pd.DataFrame(zero_data, columns = inverted_index_zonal_dictionary_second['name'].keys())
initiate_query_bow

,elimin,ant,roach,spider,killer,oz,kill,insect,spiderman,parti,...,bring,tradit,flip,indestruct,chew,proof,rip,nontox,washabl,newborn
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [179]:
zero_data = np.zeros(shape=(len(df_2),len(inverted_index_zonal_dictionary_second['name'].keys())))
initiate_bow = pd.DataFrame(zero_data, columns = inverted_index_zonal_dictionary_second['name'].keys())
initiate_bow

,elimin,ant,roach,spider,killer,oz,kill,insect,spiderman,parti,...,bring,tradit,flip,indestruct,chew,proof,rip,nontox,washabl,newborn
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
859,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
def BOW(doc_database, bow):
    terms_dictionary_for_zone = {
            'name': { },
            # 'type_of': { },
            # 'comments': { }
            }
    L2_norm_doc = []
    for zone in doc_database.columns:
        # only doing indexing for the chosen zones above
        if zone in terms_dictionary_for_zone.keys():
            # saving docID and the row content for the chosen zones
            for docID, doc in enumerate(df_2[zone]): # int, str; list[(int, str), ]
                 
                # tokenises the words in the row of the selected zone based on NLTK vocab
                sentence_doc = word_tokenize(str(doc)) # list[str,];
                # stems all words in doc
                sentence_str_1 = [p_stemmer.stem(word_tk) for word_tk in sentence_doc]  # list[str,]
                # joins the stemmed tokens back into a sentence for further procesing
                sentence_str_1 = " ".join(sentence_str_1) # str

                # subs all symbols/punctuations (including space) with a space (' '); excludes digits
                sentence_str_1 = re.sub(r'\W+', ' ',sentence_str_1)
                # subs all digits with a space (' '); a sequence of digits (without space) is given one (' ')
                sentence_str_1 = re.sub(r'\d+', ' ',sentence_str_1)
                # subs all symbols AND digits (including spaces) with a space (' '); includes digits
                sentence_str_1 = re.sub(r'[^a-zA-Z]', ' ',sentence_str_1)
                # compiles a pattern and subs out the pattern in sentence_str_1 with ' '
                sentence_str_1 = re.compile('(\s*)pron(\s*)').sub(' ', sentence_str_1)

                # tokenize to clean up one more time if 're' has introduced and garbage
                sentence_doc_1 = word_tokenize(sentence_str_1) # list[str,]
                # strip removes whitespace at beginning and end of string (not in between)
                sentence_str_2 = [str(word_tk_1).strip() for word_tk_1 in sentence_doc_1] # list[str,]
                # strings with >=2 chars only
                sentence_str_2  = [word_str for word_str in sentence_str_2 if len(word_str) >= 2] # list[str,]
                # strings which are not symbols/punctuations
                sentence_str_2 = [word_str for word_str in sentence_str_2  if word_str not in punctuation]
                # strings which are not stop-words; check with: Spacy_Eng.vocab['word'].is_stop
                sentence_str_2 = [word_str for word_str in sentence_str_2 if word_str not in stop_words]

                for term in sentence_str_2: # 'str'
                    # for each zone the term increased in doc freq and appends the posting list if term exists
                    if term in bow.columns:
                        bow.loc[docID, term] += 1
                L2_norm_doc.append(LA.norm(bow.loc[docID, :].to_numpy()))
                


                
    return bow, L2_norm_doc
      

In [181]:
def query_BOW(doc, q_bow):
                 
    # tokenises the words in the row of the selected zone based on NLTK vocab
    sentence_doc = word_tokenize(str(doc)) # list[str,];
    # stems all words in doc
    sentence_str_1 = [p_stemmer.stem(word_tk) for word_tk in sentence_doc]  # list[str,]
    # joins the stemmed tokens back into a sentence for further procesing
    sentence_str_1 = " ".join(sentence_str_1) # str

    # subs all symbols/punctuations (including space) with a space (' '); excludes digits
    sentence_str_1 = re.sub(r'\W+', ' ',sentence_str_1)
    # subs all digits with a space (' '); a sequence of digits (without space) is given one (' ')
    sentence_str_1 = re.sub(r'\d+', ' ',sentence_str_1)
    # subs all symbols AND digits (including spaces) with a space (' '); includes digits
    sentence_str_1 = re.sub(r'[^a-zA-Z]', ' ',sentence_str_1)
    # compiles a pattern and subs out the pattern in sentence_str_1 with ' '
    sentence_str_1 = re.compile('(\s*)pron(\s*)').sub(' ', sentence_str_1)

    # tokenize to clean up one more time if 're' has introduced and garbage
    sentence_doc_1 = word_tokenize(sentence_str_1) # list[str,]
    # strip removes whitespace at beginning and end of string (not in between)
    sentence_str_2 = [str(word_tk_1).strip() for word_tk_1 in sentence_doc_1] # list[str,]
    # strings with >=2 chars only
    sentence_str_2  = [word_str for word_str in sentence_str_2 if len(word_str) >= 2] # list[str,]
    # strings which are not symbols/punctuations
    sentence_str_2 = [word_str for word_str in sentence_str_2  if word_str not in punctuation]
    # strings which are not stop-words; check with: Spacy_Eng.vocab['word'].is_stop
    sentence_str_2 = [word_str for word_str in sentence_str_2 if word_str not in stop_words]

    for term in sentence_str_2: # 'str'
        # for each zone the term increased in doc freq and appends the posting list if term exists
        if term in bow.columns:
            q_bow.loc[0, term] += 1
    
    L2_norm_query = LA.norm(q_bow.to_numpy())
    return q_bow, L2_norm_query
      

In [182]:
bow, L2_norm_doc = BOW(df_2, initiate_bow)
bow

,elimin,ant,roach,spider,killer,oz,kill,insect,spiderman,parti,...,bring,tradit,flip,indestruct,chew,proof,rip,nontox,washabl,newborn
0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
859,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
860,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [183]:
L2_norm_doc

[3.3166247903554,
 2.0,
 2.6457513110645907,
 3.3166247903554,
 2.8284271247461903,
 2.8284271247461903,
 2.8284271247461903,
 4.242640687119285,
 3.1622776601683795,
 3.3166247903554,
 2.0,
 2.449489742783178,
 3.605551275463989,
 2.0,
 3.4641016151377544,
 3.0,
 3.0,
 3.605551275463989,
 2.8284271247461903,
 3.1622776601683795,
 5.0,
 2.449489742783178,
 2.23606797749979,
 2.23606797749979,
 2.23606797749979,
 3.605551275463989,
 4.58257569495584,
 3.1622776601683795,
 2.6457513110645907,
 4.358898943540674,
 3.4641016151377544,
 2.8284271247461903,
 2.23606797749979,
 5.744562646538029,
 5.196152422706632,
 5.0,
 3.0,
 3.1622776601683795,
 2.6457513110645907,
 2.23606797749979,
 2.23606797749979,
 5.656854249492381,
 2.449489742783178,
 2.449489742783178,
 4.47213595499958,
 2.6457513110645907,
 2.23606797749979,
 2.23606797749979,
 3.0,
 2.6457513110645907,
 3.4641016151377544,
 3.4641016151377544,
 3.872983346207417,
 3.4641016151377544,
 5.0990195135927845,
 3.1622776601683795,
 

In [184]:
query_bow, L2_norm_query  = query_BOW(query_1, initiate_query_bow)
query_bow

,elimin,ant,roach,spider,killer,oz,kill,insect,spiderman,parti,...,bring,tradit,flip,indestruct,chew,proof,rip,nontox,washabl,newborn
0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [185]:
L2_norm_query

3.3166247903554

In [186]:
L2_reciprocal = np.multiply(L2_norm_query,L2_norm_doc)

In [187]:
L2_reciprocal

array([11.        ,  6.63324958,  8.77496439, 11.        ,  9.38083152,
        9.38083152,  9.38083152, 14.07124728, 10.48808848, 11.        ,
        6.63324958,  8.1240384 , 11.95826074,  6.63324958, 11.48912529,
        9.94987437,  9.94987437, 11.95826074,  9.38083152, 10.48808848,
       16.58312395,  8.1240384 ,  7.41619849,  7.41619849,  7.41619849,
       11.95826074, 15.19868415, 10.48808848,  8.77496439, 14.45683229,
       11.48912529,  9.38083152,  7.41619849, 19.05255888, 17.23368794,
       16.58312395,  9.94987437, 10.48808848,  8.77496439,  7.41619849,
        7.41619849, 18.76166304,  8.1240384 ,  8.1240384 , 14.83239697,
        8.77496439,  7.41619849,  7.41619849,  9.94987437,  8.77496439,
       11.48912529, 11.48912529, 12.84523258, 11.48912529, 16.91153453,
       10.48808848, 11.        ,  6.63324958, 10.48808848, 13.67479433,
       11.        , 10.48808848, 12.40967365,  8.77496439, 11.95826074,
        9.38083152, 10.48808848, 15.90597372,  9.94987437,  8.77

In [188]:
L2_product = 1/L2_reciprocal
L2_product

array([0.09090909, 0.15075567, 0.11396058, 0.09090909, 0.10660036,
       0.10660036, 0.10660036, 0.07106691, 0.09534626, 0.09090909,
       0.15075567, 0.12309149, 0.0836242 , 0.15075567, 0.08703883,
       0.10050378, 0.10050378, 0.0836242 , 0.10660036, 0.09534626,
       0.06030227, 0.12309149, 0.13483997, 0.13483997, 0.13483997,
       0.0836242 , 0.06579517, 0.09534626, 0.11396058, 0.06917145,
       0.08703883, 0.10660036, 0.13483997, 0.05248639, 0.05802589,
       0.06030227, 0.10050378, 0.09534626, 0.11396058, 0.13483997,
       0.13483997, 0.05330018, 0.12309149, 0.12309149, 0.06741999,
       0.11396058, 0.13483997, 0.13483997, 0.10050378, 0.11396058,
       0.08703883, 0.08703883, 0.07784989, 0.08703883, 0.05913124,
       0.09534626, 0.09090909, 0.15075567, 0.09534626, 0.07312724,
       0.09090909, 0.09534626, 0.0805823 , 0.11396058, 0.0836242 ,
       0.10660036, 0.09534626, 0.06286946, 0.10050378, 0.11396058,
       0.09090909, 0.11396058, 0.10660036, 0.0805823 , 0.07312

In [189]:
query_matrix = query_bow.to_numpy()

In [190]:
doc_matrix = bow.to_numpy()

In [191]:
dot_pdt = np.dot(doc_matrix, query_matrix.T)
dot_pdt = dot_pdt.flatten()
dot_pdt

array([11.,  0.,  4.,  6.,  6.,  6.,  6.,  6.,  3.,  2.,  2.,  2.,  4.,
        2.,  7.,  2.,  4.,  0.,  2.,  4.,  6.,  2.,  3.,  0.,  2.,  4.,
        8.,  4.,  2.,  2.,  5.,  2.,  2.,  3.,  6.,  5.,  2.,  2.,  0.,
        0.,  2.,  2.,  0.,  4.,  4.,  4.,  2.,  2.,  2.,  2.,  2.,  0.,
        2.,  0.,  2.,  2.,  2.,  0.,  0.,  4.,  2.,  2.,  2.,  2.,  0.,
        2.,  2.,  6.,  2.,  2.,  2.,  2.,  2.,  2.,  4.,  2.,  4.,  2.,
        0.,  0.,  0.,  0.,  3.,  0.,  0.,  2.,  0.,  2.,  0.,  0.,  2.,
        0.,  2.,  2.,  2.,  4.,  0.,  2.,  2.,  2.,  2.,  6.,  2.,  4.,
        0.,  5.,  2.,  4.,  2.,  2.,  2.,  0.,  0.,  2.,  4.,  0.,  2.,
        2.,  6.,  0.,  0.,  0.,  0.,  0.,  3.,  2.,  2.,  2.,  2.,  6.,
        2.,  3.,  2.,  0.,  2.,  2.,  2.,  4.,  6.,  5.,  2.,  6.,  0.,
        4.,  4.,  2.,  8.,  2.,  2.,  2.,  2.,  7.,  2.,  0.,  2.,  4.,
        2.,  2.,  2.,  2.,  4.,  2.,  3.,  6.,  0.,  3.,  2.,  4.,  0.,
        2.,  3.,  1.,  4.,  0.,  7.,  2.,  3.,  4.,  4.,  4.,  2

In [192]:
cos_similarity = np.multiply(dot_pdt, L2_product)
cos_similarity = cos_similarity.tolist()
cos_similarity = enumerate(cos_similarity)
cos_similarity = dict(cos_similarity)
cos_similarity = dict(sorted(cos_similarity.items(), key = lambda x : x[1], reverse= True))
cos_similarity

{0: 1.0,
 101: 0.6837634587578276,
 449: 0.6741998624632421,
 306: 0.6446583712203042,
 339: 0.6446583712203042,
 729: 0.6446583712203042,
 4: 0.6396021490668312,
 5: 0.6396021490668312,
 6: 0.6396021490668312,
 14: 0.6092717958449425,
 603: 0.5720775535473555,
 845: 0.5533715710928597,
 3: 0.5454545454545454,
 332: 0.5454545454545454,
 829: 0.533001790889026,
 26: 0.5263613559678152,
 163: 0.5017452060042544,
 43: 0.492365963917331,
 186: 0.492365963917331,
 118: 0.48349377841522817,
 129: 0.48349377841522817,
 141: 0.48349377841522817,
 174: 0.4605661864718383,
 257: 0.4605661864718383,
 2: 0.4558423058385518,
 45: 0.4558423058385518,
 396: 0.4558423058385518,
 730: 0.45226701686664544,
 794: 0.45226701686664544,
 797: 0.45226701686664544,
 151: 0.44997704257325744,
 307: 0.4400862294233521,
 541: 0.4387634544762784,
 638: 0.4387634544762784,
 30: 0.4351941398892446,
 105: 0.4351941398892446,
 139: 0.4351941398892446,
 7: 0.4264014327112209,
 612: 0.4264014327112209,
 137: 0.42640143